In [231]:
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler  


In [232]:
# data from three different sources use different names for teams. 
# manually making a dictionary to rename teams.
team_dict = {
            'Arsenal FC': 'Arsenal',
            'Burnley FC': 'Burnley',
            'Liverpool FC': 'Liverpool',
            'Everton FC': 'Everton',
            'Fulham FC': 'Fulham',
            'Southampton FC': 'Southampton',
            'Chelsea FC': 'Chelsea',
            'West Bromwich Albion': 'West Bromwich',
            'Manchester United': 'Manchester Utd',
            'Man United': 'Manchester Utd',
            'Man City': 'Manchester City', 
            'Newcastle': 'Newcastle Utd',
            'Newcastle United': 'Newcastle Utd',
            'West Ham United': 'West Ham Utd', 
            'Tottenham Hotspur': 'Tottenham',
            'Queens Park Rangers': 'QP Rangers',
            'Watford FC': 'Watford',
            'AFC Bournemouth': 'Bournemouth',
            'Sunderland AFC': 'Sunderland',
            'Middlesbrough FC': 'Middlesbrough',
            'Brighton & Hove Albion': 'Brighton & Hove',
            'Huddersfield Town': 'Huddersfield',
            'QPR': 'QP Rangers',
            'Cardiff': 'Cardiff City',
            'Hull': 'Hull City',
            'Norwich': 'Norwich City',
            'Stoke': 'Stoke City',
            'Swansea': 'Swansea City',
            'West Brom': 'West Bromwich',
            'West Ham': 'West Ham Utd',
            'Reading FC': 'Reading',
            'Leicester': 'Leicester City'
            }

In [233]:
# Current season Roster Power Index(RPI) from PES Database.
# Each data is used to predict each season.
# For example, use pes14 to predict (train) 14/15 season.
# Merge PES 15 data with epl_table_1314 table in order to predict EPL 14/15 season.
pes14 = pd.read_csv('./Data/PES14.csv')
pes15 = pd.read_csv('./Data/PES15.csv')
pes16 = pd.read_csv('./Data/PES16.csv')
pes17 = pd.read_csv('./Data/PES17.csv')
pes18 = pd.read_csv('./Data/PES18.csv')
pes19 = pd.read_csv('./Data/PES19.csv')

In [234]:
# Past season RPIs and Match Power Index(MPI)from past season.
epl_table_1213 = pd.read_csv('./Data/epl_table_1213.csv')
epl_table_1314 = pd.read_csv('./Data/epl_table_1314.csv')
epl_table_1415 = pd.read_csv('./Data/epl_table_1415.csv')
epl_table_1516 = pd.read_csv('./Data/epl_table_1516.csv')
epl_table_1617 = pd.read_csv('./Data/epl_table_1617.csv')
epl_table_1718 = pd.read_csv('./Data/epl_table_1718.csv')

In [235]:
# Merge two RPIs.
# Merge PES 15 database and EPL 13/14 table to predict EPL 14/15
pr_14 = pd.merge(pes14, epl_table_1213, on='Team', how='outer').sort_values(by='Team').dropna()
pr_15 = pd.merge(pes15, epl_table_1314, on='Team', how='outer').sort_values(by='Team').dropna()
pr_16 = pd.merge(pes16, epl_table_1415, on='Team', how='outer').sort_values(by='Team').dropna()
pr_17 = pd.merge(pes17, epl_table_1516, on='Team', how='outer').sort_values(by='Team').dropna()
pr_18 = pd.merge(pes18, epl_table_1617, on='Team', how='outer').sort_values(by='Team').dropna()
pr_19 = pd.merge(pes19, epl_table_1718, on='Team', how='outer').sort_values(by='Team').dropna()

In [236]:
# Load EPL fixtures.
epl_detail_1314 = pd.read_csv('./Data/epl1314.csv')
epl_detail_1415 = pd.read_csv('./Data/epl1415.csv')
epl_detail_1516 = pd.read_csv('./Data/epl1516.csv')
epl_detail_1617 = pd.read_csv('./Data/epl1617.csv')
epl_detail_1718 = pd.read_csv('./Data/epl1718.csv')
epl_detail_1819 = pd.read_csv('./Data/epl1819.csv')

In [237]:
# Create columns to merge with EPL fixture dataframes.
new_cols = ['HtOff', 'HtDef', 'AtOff', 'AtDef', 'HtPesOvr', 'AtPesOvr', 
            'HtPesDef', 'HtPesMid', 'HtPesOff', 'AtPesDef', 'AtPesMid', 'AtPesOff',
            'HtPesPhy', 'HtPesSpd', 'AtPesPhy', 'AtPesSpd', 'HtDis', 'AtDis', 'HtCs', 'AtCs']

def detail_df(epl_detail_data):
    '''
    Take Home, away teams and the result from fixture dataframes.
    '''
    epl_detail_data = epl_detail_data[['HomeTeam', 'AwayTeam', 'FTR']]
    epl_detail_data = epl_detail_data.reindex(columns=epl_detail_data.columns.tolist() + new_cols)
    epl_detail_data = epl_detail_data.replace(team_dict)
    
    return epl_detail_data

epl_detail_1314 = detail_df(epl_detail_1314)
epl_detail_1415 = detail_df(epl_detail_1415)
epl_detail_1516 = detail_df(epl_detail_1516)
epl_detail_1617 = detail_df(epl_detail_1617)
epl_detail_1718 = detail_df(epl_detail_1718)
epl_detail_1819 = detail_df(epl_detail_1819)

In [238]:
def fill_df(epl_detail_data, pes_data):
    '''
    Pass RPIs to fixture dataframe.
    IndexError will be ignored.
    The model ignores relegation and promotion of teams.
    '''
    for team in epl_detail_data['HomeTeam']:
        try:
            ht_off = pes_data.loc[pes_data['Team'] == team, 'H_Att'].values[0] 
            ht_def = pes_data.loc[pes_data['Team'] == team, 'H_Def'].values[0]
            at_off = pes_data.loc[pes_data['Team'] == team, 'A_Att'].values[0]
            at_def = pes_data.loc[pes_data['Team'] == team, 'A_Def'].values[0]
            
            pes_ovr = pes_data.loc[pes_data['Team'] == team, 'Ovr'].values[0]
            pes_off = pes_data.loc[pes_data['Team'] == team, 'Fwd'].values[0]
            pes_def = pes_data.loc[pes_data['Team'] == team, 'Def'].values[0]
            pes_mid = pes_data.loc[pes_data['Team'] == team, 'Mid'].values[0]
            pes_spd = pes_data.loc[pes_data['Team'] == team, 'Spd'].values[0]
            pes_phy = pes_data.loc[pes_data['Team'] == team, 'Phy'].values[0]
            
            dis = pes_data.loc[pes_data['Team'] == team, 'DIS'].values[0]
            cs = pes_data.loc[pes_data['Team'] == team, 'CS'].values[0]

            epl_detail_data.loc[epl_detail_data['HomeTeam'] == team, 'HtDis'] = dis
            epl_detail_data.loc[epl_detail_data['HomeTeam'] == team, 'HtCs'] = cs
            epl_detail_data.loc[epl_detail_data['AwayTeam'] == team, 'AtDis'] = dis
            epl_detail_data.loc[epl_detail_data['AwayTeam'] == team, 'AtCs'] = cs
            
            epl_detail_data.loc[epl_detail_data['HomeTeam'] == team, 'HtOff'] = ht_off
            epl_detail_data.loc[epl_detail_data['HomeTeam'] == team, 'HtDef'] = ht_def
            epl_detail_data.loc[epl_detail_data['AwayTeam'] == team, 'AtOff'] = at_off
            epl_detail_data.loc[epl_detail_data['AwayTeam'] == team, 'AtDef'] = at_def
            
            epl_detail_data.loc[epl_detail_data['HomeTeam'] == team, 'HtPesOvr'] = pes_ovr
            epl_detail_data.loc[epl_detail_data['AwayTeam'] == team, 'AtPesOvr'] = pes_ovr
            epl_detail_data.loc[epl_detail_data['HomeTeam'] == team, 'HtPesOff'] = pes_off
            epl_detail_data.loc[epl_detail_data['AwayTeam'] == team, 'AtPesOff'] = pes_off
            epl_detail_data.loc[epl_detail_data['HomeTeam'] == team, 'HtPesDef'] = pes_def
            epl_detail_data.loc[epl_detail_data['AwayTeam'] == team, 'AtPesDef'] = pes_def
            epl_detail_data.loc[epl_detail_data['HomeTeam'] == team, 'HtPesMid'] = pes_mid
            epl_detail_data.loc[epl_detail_data['AwayTeam'] == team, 'AtPesMid'] = pes_mid
            epl_detail_data.loc[epl_detail_data['HomeTeam'] == team, 'HtPesSpd'] = pes_spd
            epl_detail_data.loc[epl_detail_data['AwayTeam'] == team, 'AtPesSpd'] = pes_spd
            epl_detail_data.loc[epl_detail_data['HomeTeam'] == team, 'HtPesPhy'] = pes_phy
            epl_detail_data.loc[epl_detail_data['AwayTeam'] == team, 'AtPesPhy'] = pes_phy
        
        except IndexError:
            pass
        
    return epl_detail_data
            
epl_concat_1314 = fill_df(epl_detail_1314, pr_14)
epl_concat_1415 = fill_df(epl_detail_1415, pr_15)
epl_concat_1516 = fill_df(epl_detail_1516, pr_16)
epl_concat_1617 = fill_df(epl_detail_1617, pr_17)
epl_concat_1718 = fill_df(epl_detail_1718, pr_18)
epl_concat_1819 = fill_df(epl_detail_1819, pr_19)

In [239]:
# Final model.
model_df = pd.concat([epl_concat_1314, epl_concat_1415, epl_concat_1516, epl_concat_1617, epl_concat_1718])

# Drop NAs from promotion/relegation.
model_df = model_df.dropna()

In [240]:
X = model_df[new_cols]
y = model_df['FTR']

In [323]:
scaler = StandardScaler()  
X = scaler.fit_transform(X)

knn = KNeighborsClassifier(n_neighbors=8)
lr = LogisticRegression(random_state=42)
rf = RandomForestClassifier(random_state=42)
mlp = MLPClassifier(hidden_layer_sizes=(2),solver='sgd',learning_rate_init= 0.01, random_state=42)
 
knn.fit(X, y)
lr.fit(X, y)
rf.fit(X, y)
mlp.fit(X, y);

In [324]:
print('---- Score on Train (Past 5 Seasons) ----')
print('KNN:' + str(knn.score(X, y)))
print('LR:' + str(lr.score(X, y)))
print('RF:' + str(rf.score(X, y)))
print('MLP:' + str(mlp.score(X, y)))

print('---- Score on Test (Current Season) ----')
print('KNN:' + str(knn.score(epl_detail_1819.dropna()[new_cols], epl_detail_1819.dropna()['FTR'])))
print('LR:' + str(lr.score(epl_detail_1819.dropna()[new_cols], epl_detail_1819.dropna()['FTR'])))
print('RF:' + str(rf.score(epl_detail_1819.dropna()[new_cols], epl_detail_1819.dropna()['FTR'])))
print('MLP:' + str(mlp.score(epl_detail_1819.dropna()[new_cols], epl_detail_1819.dropna()['FTR'])))

---- Score on Train (Past 5 Seasons) ----
KNN:0.5698529411764706
LR:0.5308823529411765
RF:0.9801470588235294
MLP:0.5264705882352941
---- Score on Test (Current Season) ----
KNN:0.5813953488372093
LR:0.4418604651162791
RF:0.37209302325581395
MLP:0.4186046511627907


In [325]:
def get_result(df, data):
    if knn.predict_proba(data)[0] > 0.52:
        df['Predict'] = 'A'
    elif knn.predict_proba(data)[2] > 0.52:
        df['Predict'] = 'H'
    else:
        df['Predict'] = 'D'
test = pd.DataFrame(columns=['Predict'])
prob = knn.predict_proba(predict_X)
for i in range(len(prob)):
    if (prob[i][0] == prob[i][2]) | (abs(prob[i][0] - prob[i][2]) < 0.05):
        test.loc[i, 'Predict'] = 'D'
    elif prob[i].max() == prob[i][0]:
        test.loc[i, 'Predict'] = 'A'
    elif prob[i].max() == prob[i][2]:
        test.loc[i, 'Predict'] = 'H'
    else:
        test.loc[i, 'Predict'] = 'D'

In [326]:
test
knn.predict_proba(predict_X).round(4)
#test['Predict'] == y['FTR']
#(y.reset_index()['FTR'] == test['Predict']).mean()
#(compare.reset_index()['Actual'] == test['Predict']).mean()
#test['Predict']
#(knn.predict(X)=='D').sum()
#(y=='D').sum() #339

array([[0.25 , 0.125, 0.625],
       [0.375, 0.25 , 0.375],
       [0.375, 0.25 , 0.375],
       [0.25 , 0.25 , 0.5  ],
       [0.25 , 0.125, 0.625],
       [0.25 , 0.125, 0.625],
       [0.25 , 0.125, 0.625],
       [0.25 , 0.125, 0.625],
       [0.25 , 0.125, 0.625],
       [0.375, 0.125, 0.5  ],
       [0.25 , 0.125, 0.625],
       [0.25 , 0.25 , 0.5  ],
       [0.25 , 0.25 , 0.5  ],
       [0.375, 0.25 , 0.375],
       [0.25 , 0.25 , 0.5  ],
       [0.375, 0.25 , 0.375],
       [0.375, 0.125, 0.5  ],
       [0.375, 0.125, 0.5  ],
       [0.25 , 0.125, 0.625],
       [0.25 , 0.125, 0.625],
       [0.25 , 0.125, 0.625],
       [0.25 , 0.25 , 0.5  ],
       [0.25 , 0.125, 0.625],
       [0.375, 0.25 , 0.375],
       [0.375, 0.25 , 0.375],
       [0.25 , 0.25 , 0.5  ],
       [0.375, 0.25 , 0.375],
       [0.375, 0.25 , 0.375],
       [0.25 , 0.125, 0.625],
       [0.375, 0.25 , 0.375],
       [0.25 , 0.125, 0.625],
       [0.375, 0.125, 0.5  ],
       [0.25 , 0.125, 0.625],
       [0.

In [327]:
epl_detail_1819.dropna(inplace=True)
predict_X = epl_detail_1819[new_cols]
compare = epl_detail_1819[['HomeTeam', 'AwayTeam']]
compare = compare.reindex(columns=compare.columns.tolist() + ['Actual', 'Predict'])
compare['Actual'] = epl_detail_1819['FTR']
compare['Predict'] = knn.predict(predict_X)

In [322]:
compare

,HomeTeam,AwayTeam,Actual,Predict
0,Manchester Utd,Leicester City,H,H
3,Huddersfield,Chelsea,A,A
4,Newcastle Utd,Tottenham,A,A
7,Arsenal,Manchester City,A,H
8,Liverpool,West Ham Utd,H,H
9,Southampton,Burnley,D,H
11,Chelsea,Arsenal,H,H
12,Everton,Southampton,H,H
15,West Ham Utd,Bournemouth,A,H
17,Burnley,Watford,A,A


In [252]:
compare[(compare['AwayTeam'] == 'Liverpool') | (compare['HomeTeam'] == 'Liverpool')]

,HomeTeam,AwayTeam,Actual,Predict
8,Liverpool,West Ham Utd,H,H
19,Crystal Palace,Liverpool,A,A
34,Leicester City,Liverpool,A,A
45,Tottenham,Liverpool,A,A
56,Liverpool,Southampton,H,H
61,Chelsea,Liverpool,D,A


In [253]:
compare[(compare['AwayTeam'] == 'Manchester Utd') | (compare['HomeTeam'] == 'Manchester Utd')]

,HomeTeam,AwayTeam,Actual,Predict
0,Manchester Utd,Leicester City,H,H
29,Manchester Utd,Tottenham,A,H
37,Burnley,Manchester Utd,A,A
46,Watford,Manchester Utd,A,A
66,West Ham Utd,Manchester Utd,H,A


In [257]:
compare[(compare['AwayTeam'] == 'Chelsea') | (compare['HomeTeam'] == 'Chelsea')]

,HomeTeam,AwayTeam,Actual,Predict
3,Huddersfield,Chelsea,A,A
11,Chelsea,Arsenal,H,H
27,Newcastle Utd,Chelsea,A,A
31,Chelsea,Bournemouth,H,H
59,West Ham Utd,Chelsea,D,A
61,Chelsea,Liverpool,D,A
